install transformers library

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-qzx_33do
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qzx_33do
  Resolved https://github.com/huggingface/transformers.git to commit acd653164b6874e395ac9d46850f67599d8cdb58
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8148049 sha256=aaaaae8c09cd8dfb944fa7b304372944977f6e0168d2cba727af416c937ed0e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-9zfj96yk/wheels/cf/59/82/6492402e887a68975030bf8c06532260abc16abb7ccd8127cc
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/mjf-su/VLM-SAD

Cloning into 'VLM-SAD'...
remote: Enumerating objects: 5450, done.
remote: Counting objects: 100% (864/864), done.
remote: Compressing objects: 100% (347/347), done.
remote: Total 5450 (delta 555), reused 794 (delta 488), pack-reused 4586
Receiving objects: 100% (5450/5450), 893.71 MiB | 30.74 MiB/s, done.
Resolving deltas: 100% (560/560), done.
Updating files: 100% (4659/4659), done.


In [4]:
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
import os
import numpy as np
import ipdb
from torch.utils.tensorboard import SummaryWriter
import json

load BLIP model and processor (image processor, tokenizer)

In [5]:
# model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [6]:
# Ensure blip model loaded correctly
# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# question = "how many dogs are in the picture?"
# inputs = processor(raw_image, question, return_tensors="pt")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

define VQA container. need to overload len() and get_item()

In [7]:
class VQADataset(torch.utils.data.Dataset):
  def __init__(self, questions, answers, image_paths, processor):
    self.questions = questions
    self.answers = answers
    self.image_paths = image_paths
    self.processor = processor

    root = !pwd
    self.data_path = os.path.join(root[0], "VLM-SAD", "full_experimentation", "Fine-tuning", "data")


  def __len__(self):
    return len(self.questions)

  def __getitem__(self, idx):
    # get image + text
    question = self.questions[idx]
    answer = self.answers[idx]
    image = Image.open(os.path.join(self.data_path, self.image_paths[idx])).convert("RGB")
    text = question

    encoding = self.processor(image, text, padding = "max_length", truncation = True, return_tensors = "pt")
    labels = self.processor.tokenizer.encode(
        answer, padding = 'max_length', return_tensors = "pt"
    )

    encoding["labels"] = labels

    for k, v in encoding.items(): encoding[k] = v.squeeze()

    return encoding

In [8]:
from torch.utils.data import DataLoader
from tqdm import tqdm

build train and test set division

In [9]:
folder = !pwd
folder = folder[0]
print(folder)

# train on first half of experiments, test on second half
anom_train_dataset_exp = {"anomaly_stop_sign": ["exp_1", "exp_2", "exp_3", "exp_4", "exp_5", "exp_6", "exp_7", "exp_8", "exp_9", "exp_10"]}
nom_train_dataset_exp = {"nominal_traffic_light": ["exp_1", "exp_2", "exp_3", "exp_4", "exp_5"],
                     "nominal_stop_sign": ["exp_1", "exp_2", "exp_3", "exp_4", "exp_5"]}
anom_test_dataset_exp = {"anomaly_stop_sign": ["exp_11", "exp_12", "exp_13", "exp_14", "exp_15", "exp_16"]}
nom_test_dataset_exp = {"nominal_traffic_light": ["exp_6", "exp_7", "exp_8", "exp_9", "exp_10"],
                        "nominal_stop_sign": ["exp_6", "exp_7", "exp_8", "exp_9", "exp_10"]}

anom_train_questions = []
anom_train_answers = []
anom_train_image_paths = []

nom_train_questions = []
nom_train_answers = []
nom_train_image_paths = []

anom_test_questions = []
anom_test_answers = []
anom_test_image_paths = []

nom_test_questions = []
nom_test_answers = []
nom_test_image_paths = []


for scenario in anom_train_dataset_exp.keys(): # anomalies in training set
  for exp in anom_train_dataset_exp[scenario]:
    exp_folder = os.path.join(folder, "VLM-SAD", "full_experimentation", "Fine-tuning", "data", scenario, exp)
    experiment = np.load(os.path.join(exp_folder, "trim_anom_VQADataset.npz"))

    anom_train_questions += experiment["questions"].tolist()
    anom_train_answers += experiment["answers"].tolist()
    anom_train_image_paths += experiment["filenames"].tolist()

for scenario in nom_train_dataset_exp.keys(): # nominal samples in training set
  for exp in nom_train_dataset_exp[scenario]:
    exp_folder = os.path.join(folder, "VLM-SAD", "full_experimentation", "Fine-tuning", "data", scenario, exp)
    experiment = np.load(os.path.join(exp_folder, "trim_nom_VQADataset.npz"))

    nom_train_questions += experiment["questions"].tolist()
    nom_train_answers += experiment["answers"].tolist()
    nom_train_image_paths += experiment["filenames"].tolist()

for scenario in anom_test_dataset_exp.keys(): # anomalies in the test set
  for exp in anom_test_dataset_exp[scenario]:
    exp_folder = os.path.join(folder, "VLM-SAD", "full_experimentation", "Fine-tuning", "data", scenario, exp)
    experiment = np.load(os.path.join(exp_folder, "trim_anom_VQADataset.npz"))

    anom_test_questions += experiment["questions"].tolist()
    anom_test_answers += experiment["answers"].tolist()
    anom_test_image_paths += experiment["filenames"].tolist()

for scenario in nom_test_dataset_exp.keys(): # nominal samples in the training set
  for exp in nom_test_dataset_exp[scenario]:
    exp_folder = os.path.join(folder, "VLM-SAD", "full_experimentation", "Fine-tuning", "data", scenario, exp)
    experiment = np.load(os.path.join(exp_folder, "trim_nom_VQADataset.npz"))

    nom_test_questions += experiment["questions"].tolist()
    nom_test_answers += experiment["answers"].tolist()
    nom_test_image_paths += experiment["filenames"].tolist()

/content


In [10]:
# form training dataset
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
anom_train_dataset = VQADataset(anom_train_questions, anom_train_answers, anom_train_image_paths, processor)
nom_train_dataset = VQADataset(nom_train_questions, nom_train_answers, nom_train_image_paths, processor)

anom_test_dataset = VQADataset(anom_test_questions, anom_test_answers, anom_test_image_paths, processor)
nom_test_dataset = VQADataset(nom_test_questions, nom_test_answers, nom_test_image_paths, processor)

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [11]:
print(len(anom_train_dataset))
print(len(nom_train_dataset))

print(len(anom_test_dataset))
print(len(nom_test_dataset))

204
414
58
391


In [12]:
torch.cuda.empty_cache()

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
# Run characterization
run_name = "nom-tl&ss_anom-ss"

# Hyperparameters
epochs = 20
lr = 5e-5
batch_size = 4
test_nums = [1,2]

assert len(nom_train_dataset) >= len(anom_train_dataset)
assert batch_size % 2 == 0

# Optimization setup
anom_train_dataloader = DataLoader(anom_train_dataset, batch_size = batch_size // 2, sampler = torch.utils.data.RandomSampler(anom_train_dataset, replacement = True, num_samples = len(nom_train_dataset)))
nom_train_dataloader = DataLoader(nom_train_dataset, batch_size = batch_size // 2, shuffle = True)
assert len(anom_train_dataloader) == len(nom_train_dataloader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

anom_test_dataloader = DataLoader(anom_test_dataset, batch_size = batch_size, shuffle = False)
nom_test_dataloader = DataLoader(nom_test_dataset, batch_size = batch_size, shuffle = False)

for test_num in test_nums:
  torch.cuda.empty_cache()
  model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

  optimizer = torch.optim.AdamW(model.parameters(), lr = lr)

  # Log learning performance
  writer = SummaryWriter(os.path.join(folder, "Tensorboard", run_name, "test" + str(test_num)))
  with open(os.path.join(folder, "Tensorboard", run_name, "test" + str(test_num), "train_config"), "w") as io:
    json.dump(nom_train_dataset_exp, io)
    json.dump(anom_train_dataset_exp, io)
  with open(os.path.join(folder, "Tensorboard", run_name, "test" + str(test_num), "nom_test_config"), "w") as io:
    json.dump(nom_test_dataset_exp, io)
  with open(os.path.join(folder, "Tensorboard", run_name, "test" + str(test_num), "anom_test_config"), "w") as io:
    json.dump(anom_test_dataset_exp, io)

  model.to(device)
  model.train()

  for epoch in range(epochs):
    print("Epoch: ", epoch)
    epoch_train_loss = 0

    anom_train_iter = iter(anom_train_dataloader)
    nom_train_iter = iter(nom_train_dataloader)

    for idx, (anom_batch, nom_batch) in tqdm(enumerate(zip(anom_train_iter, nom_train_iter))): # training loop
      input_ids = torch.cat((anom_batch["input_ids"], nom_batch["input_ids"])).to(device)
      pixel_values = torch.cat((anom_batch["pixel_values"], nom_batch["pixel_values"])).to(device)
      attention_mask = torch.cat((anom_batch["attention_mask"], nom_batch["attention_mask"])).to(device)
      labels = torch.cat((anom_batch["labels"], nom_batch["labels"])).to(device)

      optimizer.zero_grad()
      outputs = model(input_ids = input_ids,
                      pixel_values = pixel_values,
                      attention_mask = attention_mask,
                      labels = labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step() # original tutorial had .zero_grad() after this

      writer.add_scalar("train_iter_loss", loss.item(), idx + epoch*len(nom_train_dataloader)) # loss per epoch (is it mean? --> need equation)
      epoch_train_loss += loss.item()
    writer.add_scalar("train_epoch_loss", epoch_train_loss / len(nom_train_dataloader), epoch)
    print("Epoch loss: ", epoch_train_loss / len(nom_train_dataloader))

    correct_cls = 0
    for batch in anom_test_dataloader: # evaluation loop: accuracy
      input_ids = batch["input_ids"].to(device)
      pixel_values = batch["pixel_values"].to(device)
      attention_mask = batch["attention_mask"].to(device)

      generated_output = model.generate(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, max_length = 300)
      generated_caption = processor.batch_decode(generated_output, skip_special_tokens = True)
      for caption in generated_caption:
        cls_list = []
        for sentence in caption.split("."):
          if "classification" not in sentence:
            continue
          else:
            if "normal" in sentence:
              cls_list.append(False)
            elif "anomaly" in sentence:
              cls_list.append(True)
        if np.any(cls_list):
          correct_cls += 1
    writer.add_scalar("anom_test_accuracy", correct_cls / len(anom_test_dataset), epoch)
    print("anom_accuracy:", correct_cls / len(anom_test_dataset))

    correct_cls = 0
    for batch in nom_test_dataloader: # evaluation loop: accuracy
      input_ids = batch["input_ids"].to(device)
      pixel_values = batch["pixel_values"].to(device)
      attention_mask = batch["attention_mask"].to(device)

      generated_output = model.generate(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, max_length = 300)
      generated_caption = processor.batch_decode(generated_output, skip_special_tokens = True)
      for caption in generated_caption:
        cls_list = []
        for sentence in caption.split("."):
          if "classification" not in sentence:
            continue
          else:
            if "normal" in sentence:
              cls_list.append(False)
            elif "anomaly" in sentence:
              cls_list.append(True)
        if len(cls_list) > 0 and np.all(np.invert(cls_list)):
          correct_cls += 1
    writer.add_scalar("nom_test_accuracy", correct_cls / len(nom_test_dataset), epoch)
    print("nom_accuracy:", correct_cls / len(nom_test_dataset))
  writer.flush()
  writer.close()

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Epoch:  0


0it [00:00, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
207it [02:41,  1.28it/s]


Epoch loss:  4.173140692826054
anom_accuracy: 1.0
nom_accuracy: 0.0
Epoch:  1


...

Epoch loss:  1.3860244036872607
anom_accuracy: 0.9137931034482759
nom_accuracy: 0.8465473145780051
Epoch:  18


207it [02:33,  1.35it/s]


Epoch loss:  1.3876461896343508
anom_accuracy: 0.7068965517241379
nom_accuracy: 0.9923273657289002
Epoch:  19


207it [02:32,  1.35it/s]


Epoch loss:  1.3919760161551877
anom_accuracy: 0.8448275862068966
nom_accuracy: 0.8670076726342711


In [15]:
# nom_test_dataloader = DataLoader(nom_test_dataset, batch_size = 4, shuffle = True)
# correct_cls = 0
# for batch in nom_test_dataloader: # evaluation loop: accuracy
#   input_ids = batch["input_ids"].to(device)
#   pixel_values = batch["pixel_values"].to(device)
#   attention_mask = batch["attention_mask"].to(device)

#   generated_output = model.generate(input_ids = input_ids, pixel_values = pixel_values, attention_mask = attention_mask, max_length = 300)
#   generated_caption = processor.batch_decode(generated_output, skip_special_tokens = True)
#   for caption in generated_caption:
#     cls_list = []
#     for sentence in caption.split("."):
#       if "classification" not in sentence:
#         continue
#       else:
#         if "normal" in sentence:
#           cls_list.append(False)
#         elif "anomaly" in sentence:
#           cls_list.append(True)
#     if np.any(cls_list):
#       correct_cls += 1
# print(correct_cls / len(nom_test_dataset))

In [16]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [17]:
!zip -r ./results.zip ./Tensorboard/

from google.colab import files
files.download("./results.zip")

  adding: Tensorboard/ (stored 0%)
  adding: Tensorboard/nom-tl&ss_anom-ss/ (stored 0%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test2/ (stored 0%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test2/anom_test_config (deflated 39%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test2/train_config (deflated 63%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test2/events.out.tfevents.1701860335.68df80e79d6f.533.1 (deflated 69%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test2/nom_test_config (deflated 52%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test1/ (stored 0%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test1/anom_test_config (deflated 39%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test1/train_config (deflated 63%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test1/events.out.tfevents.1701847263.68df80e79d6f.533.0 (deflated 69%)
  adding: Tensorboard/nom-tl&ss_anom-ss/test1/nom_test_config (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>